In [ ]:
## Imports

In [ ]:
!pip install openai
!pip install tiktoken
!pip install langchain
!pip install -U sentence-transformers
!pip install accelerate
!pip install einops
!pip install transformers
!pip install xformers
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.6 MB/s eta 0:00:0

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pickle
from langchain import PromptTemplate, LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain import HuggingFacePipeline, HuggingFaceHub
import os
import openai

Mounted at /content/drive


In [ ]:
GPT_3_KEY = ''
GPT_4_KEY = ''
OPENAI_KEY = GPT_4_KEY
HF_API_KEY = ''

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_API_KEY

In [ ]:
# Set openai.api_key to the OPENAI environment variable
openai.api_key = os.environ["OPENAI_API_KEY"]

### API Check

## Environment Variables

In [ ]:
# models
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
GPT_3 = "gpt-3.5-turbo"
GPT_4 = "gpt-4"
LOCAL_MODEL = "DrugGPT"
# Use local embedding model?
USELOCALEMBD = True
# Use local model?
USELOCALMODEL = False
# Use langchain model or build your own from transformers library?
USELANGCHAINMODEL = True
# Use huggingface model? RECOMMNAD, OTHERWISE IT"S TOO SLOW
USEHFAPI = False
# Use langchain openAI model or directly use openAI api
USEOPENAI = True

In [ ]:
# Define the slice size
SLICE_THRESHOLD = 5000
# Define the size of the evaluation set
EVALUATION_SIZE = 1000
# Define the smaller size of the evaluation set
EVALUATION_SIZE_SMALL = 500

#### Data Paths

In [ ]:
data_paths_dict = {
    'pubmedqa': {
        'type': 'binary',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'ade': {
        'type': 'text',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'chatDoctor': {
        'type': 'text',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'DDI_binary': {
        'type': 'binary',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'drug_usage': {
        'type': 'text',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'medmcqa': {
        'type': 'mc',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'mmlu_mc': {
        'type': 'mc',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'usmle_mc': {
        'type': 'mc',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    },
    'moderna_interactions': {
        'type': 'binary',
        'data': '',
        'answer': '',
        'GPT_3_answer': '',
        'GPT_4_answer': ''
    }
}


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


## Local Model

1. model: Specifies the model to be used for text generation. In this case, we will use 'DrugGPT' for DrugGPT evaluation.

2. tokenizer = AutoTokenizer.from_pretrained(model): Loads the tokenizer associated with the model. A tokenizer is responsible for converting input text into a format (like a list of tokens) that the model can understand.

4. torch_dtype=torch.bfloat16: This is the data type of the torch tensors used in the pipeline. Using bfloat16 instead of the default float32 can reduce memory usage and computation time, at the expense of some numerical precision.

5. trust_remote_code=True: This allows execution of code from the remote model, which is necessary if the model includes custom post-processing.

6. device_map="auto": This allows the model to run on the best available device (either a GPU or CPU).

7. max_length=200: This specifies the maximum length of the generated text.

8. do_sample=True: This means that the generated text will be sampled from the model's output distribution rather than just taking the most probable token at each step. This makes the output more diverse and creative.

9. top_k=10: This parameter is used for the "top-k sampling" strategy, which only considers the top k most probable tokens at each step.

10. num_return_sequences=1: This means that the model will return one generated sequence.

11. eos_token_id=tokenizer.eos_token_id: The id of the End of Sentence (EOS) token. The generation process will stop if the model generates this token.

#### Save Local Model （We use Langchain instead）

In [ ]:
if not USELANGCHAINMODEL:
  print("We are using our own local model")
  # Save the pipeline
  if os.path.isfile(local_model_path):
    with open(local_model_path, 'rb') as f:
      pipeline = pickle.load(f)
  else:
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL)
    model = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL, trust_remote_code=True)
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
    )

    with open(local_model_path, 'wb') as f:
        pickle.dump(pipeline, f)

  sequences = pipeline(
    "Tell me a joke",
      max_length=200,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )
  for seq in sequences:
      print(f"Result: {seq['generated_text']}")

### Langchain LLM Local Model or HF API

In [ ]:
if USEHFAPI:
  llm = HuggingFaceHub(repo_id=LOCAL_MODEL, task="text-generation", model_kwargs={"temperature":0.5, "max_length":2000, "trust_remote_code": True})
  print(f"We are using HF API {llm}")
else:
  llm = HuggingFacePipeline.from_model_id(model_id=LOCAL_MODEL, task="text-generation", device=0, model_kwargs={"temperature":0.5, "max_length":2000, "trust_remote_code": True})
  print(f"We are using local mod")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We are using local mod


## Chat Agent Prompts

### System Prompt

In [ ]:
SYS_PROMPT_INSTRUCTIONS = {
    "pubmedqa": {
        "task": "You are tasked with answering question based on the provided content related to pubmed articles. You should thoroughly understand the question, extract relevant information from the content, analyze this information and finally provide an answer. BE CONCISE AND ACCURATE",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer, Must be a 'yes', 'no'.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Stick to summarizing and interpreting the content as objectively and accurately as possible. Remember that you are providing an analysis based on the content and not diagnosing or treating medical conditions."
    },
    "ade": {
        "task": "Your task is to identify the adverse drug effect mentioned in the question based on the provided content . Thoroughly understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer, which should identify the adverse effect.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions."
    },

    "chatDoctor": {
        "task": "Your task is to list the medications based on the provided content related to the symptom or disease mentioned in the question. Understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer, which should be a list of medications related to the symptom or disease.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions."
    },
    "DDI_binary": {
        "task": "You are tasked to answer yes/no questions based on the provided content about whether there is a reaction between the drugs mentioned. Understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer. The answer should be 'yes' or 'no'.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions."
    },

    "drug_usage": {
        "task": "Your task is to answer two yes/no questions based on the provided content related to the usage of the drug provided. Understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer, The answer should be two 'yes' or 'no' separated by comma.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions."
    },

    "medmcqa": {
        "task": "Your task is to answer multiple choice questions based on the provided content about medication and pharmacology. Understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer, which should be a single letter in the alphabet representing the best option among the multiple choices provided in the question.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions."
    },

    "mmlu_mc": {
        "task": "Your task is to answer multiple choice questions based on the provided content about drug recommendation. Understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer, which should be a single letter in the alphabet representing the best option among the multiple choices provided in the question.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions."
    },

    "usmle_mc": {
        "task": "Your task is to answer multiple choice questions based on the provided content about drug recommendation. Understand the question, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant content.
Final Answer: Provide the final answer, which should be a single letter in the alphabet representing the best option among the multiple choices provided in the question.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the content as objectively and accurately as possible."
    },
    "moderna_interactions": {
        "task": "You are tasked to answer yes/no questions based on the provided content about whether the therapeutic efficacy of Moderna COVID-19 Vaccine decrease when used with another drug. Understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.",
        "answer_format": """
Analysis: Provide an analysis that logically leads to the answer based on the relevant information.
Final Answer: Provide the final answer. The answer should be 'yes' or 'no'.
        """,
        "not_to_dos": "Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions."
    },
}


In [ ]:
sys_template = PromptTemplate(
    input_variables=["task", "answer_format", "not_to_dos"],
    template="Task: {task}\nAnswer Format: {answer_format}\nNot to dos: {not_to_dos}\n----Below are some examples-----\n"
)

In [ ]:
def generate_sys_prompt(knowledge_set):
  instruction = SYS_PROMPT_INSTRUCTIONS[knowledge_set]
  return sys_template.format(**instruction)

### Few Shots

In [ ]:
FS_EXAMPLES = {
    "pubmedqa": [
        {
            "question": "Does intermittent warm blood cardioplegia provide adequate myocardial resuscitation after global ischaemia?",
            "content" : "",
            "analysis": "Based on the provided information from the study, intermittent warm blood cardioplegia does not seem to provide adequate myocardial resuscitation after global ischaemia.",
            "final_answer": "no"
        },
        ],
    "ade": [
        {
            "question": "Identify the adverse drug reaction related to azithromycin in this context: Intravenous azithromycin-induced ototoxicity.",
            "content": "",
            "analysis": "It can be concluded that the adverse drug reaction related to azithromycin is ototoxicity.",
            "final_answer": "ototoxicity"
        },
    ],
    "chatDoctor": [
        {
            "question": "What are the recommended medications for Panic disorder?",
            "content": "",
            "analysis": "The medications listed in the content are the recommended ones for treating Panic disorder.",
            "final_answer": "lorazepam, alprazolam, clonazepam, paroxetine, venlafaxine, mirtazapine, buspirone, fluvoxamine, imipramine, desvenlafaxine, clomipramine, acamprosate"
        },
    ],
    "DDI_binary": [
        {
            "question": "Is there a reaction between Sibutramine and Icatibant?",
            "content": "",
            "analysis": "From the given context, it can be concluded that there is a reaction effect identified between Sibutramine and Icatibant",
            "final_answer": "yes"
        },
    ],
    "drug_usage": [
        {
            "question": "Answer the following two questions about acetaminophen:\n Have studies shown adverse effects on preganancy?\n Have studies shown an interaction with alchohol?",
            "content": "",
            "analysis": "The content shows that acetaminophen has shwon adverse effects on preganancy demonstrated in animal studies. The content has shown that acetaminophen interacts with alchohol",
            "final_answer": "yes, yes"
        },
    ],
    "usmle_mc": [
        {
            "question": "A 62-year-old woman presents for a regular check-up. She complains of lightheadedness and palpitations which occur episodically. Past medical history is significant for a myocardial infarction 6 months ago and NYHA class II chronic heart failure. She also was diagnosed with grade I arterial hypertension 4 years ago. Current medications are aspirin 81 mg, atorvastatin 10 mg, enalapril 10 mg, and metoprolol 200 mg daily. Her vital signs are a blood pressure of 135/90 mm Hg, a heart rate of 125/min, a respiratory rate of 14/min, and a temperature of 36.5¬∞C (97.7¬∞F). Cardiopulmonary examination is significant for irregular heart rhythm and decreased S1 intensity. ECG is obtained and is shown in the picture (see image). Echocardiography shows a left ventricular ejection fraction of 39%. Which of the following drugs is the best choice for rate control in this patient? A:Atenolol, B:Diltiazem, C:Propafenone, D:Digoxin",
            "content": "",
            "analysis": "The 62-year-old woman's presentation of lightheadedness and palpitations, an irregular heart rhythm on cardiopulmonary examination, and her history of myocardial infarction and NYHA class II chronic heart failure suggest she may be suffering from a rate control issue related to her heart. The patient is already on a regimen that includes metoprolol, a beta-blocker used for rate control, and the dose is relatively high. This makes adding another beta-blocker like atenolol less effective. The use of calcium channel blockers, like diltiazem, might worsen her heart failure symptoms. Propafenone, a class IC antiarrhythmic, is mainly used to treat conditions that cause a fast heart rate, such as atrial fibrillation and atrial flutter, and ventricular arrhythmias, but may not be suitable for a patient with heart failure. On the other hand, digoxin has been traditionally used for rate control in patients with heart failure and atrial fibrillation, as it increases the strength of the heart's contractions and slows down the electrical impulses in the AV node.",
            "final_answer": "D",
        },
    ],
    "mmlu_mc": [
        {
            "question": "A 55-year-old man is brought to the emergency department by his wife because he told her he did not want to live anymore. During the past 6 weeks, he has experienced fatigue, loss of interest in usual activities, a 7-kg (15-lb) weight loss, and insomnia. He has no history of serious medical or psychiatric illness. Vital signs are temperature 37.0¬∞C (98.6¬∞F), pulse 80/min, respirations 16/min, and blood pressure 140/82 mm Hg. Physical examination discloses no abnormalities. Beck Depression Inventory score is 35 (severely depressed). He says he feels guilty that his investments have not done well and that he has ruined his family finances. He reports he hears voices at night telling him he has sinned. In addition to olanzapine, which of the following is the best treatment option for this patient? A:Divalproex, B:Fluoxetine, C:Lamotrigine, D:Lithium carbonate",
            "content": "",
            "analysis": "The patient presents with symptoms indicative of a major depressive episode with possible psychotic features, as indicated by his hearing voices. The severity of his depression is confirmed by his high Beck Depression Inventory score. As such, his treatment should address both the depressive symptoms and the psychotic symptoms.",
            "final_answer": "B",
        },
    ],
    "medmcqa": [
        {
            "question": "A 42year old female presents with diazepam and alcohol overdose. She is comatose. Temperature is 34.5degC. BP is 100/80 mmHg. Creatinine is 2.4mg/dL, AST -500, GGT- 35 IU. Urine dipstick showed 3+ for blood but urine analysis was normal. USG abdomen was normal. What is the most likely diagnosis? A:Hypothermia, B: Alcoholic hallucinosis, C: Rhabdomyolysis, D: Acute interstitial nephritis",
            "content": "",
            "analysis": "The patient has presented following an overdose of diazepam and alcohol. Given the symptoms and clinical findings, we are considering several potential diagnoses: Hypothermia, Alcoholic hallucinosis, Rhabdomyolysis, and Acute interstitial nephritis.Hypothermia (option A) could be suggested by the low body temperature, but it doesn't account for all of the patient's symptoms and test results.Alcoholic hallucinosis (option B) could be a possibility given the alcohol overdose, but the patient is comatose and not displaying signs of hallucinations.Acute interstitial nephritis (option D) might cause an increase in creatinine, but it generally does not result in blood in the urine without abnormality in urine analysis.Rhabdomyolysis (option C), on the other hand, can result from a drug overdose. This condition causes muscle breakdown, which releases myoglobin into the bloodstream. Myoglobin can cause renal failure, which could explain the increased creatinine. It also can show up as blood on a urine dipstick test while other aspects of a urine analysis remain normal, due to the fact that common urine dipstick tests cannot differentiate between myoglobin and hemoglobin.",
            "final_answer": "C",
        },
    ],
    "moderna_interactions": [
        {
            "question": "Does the therapeutic efficacy of Moderna COVID-19 Vaccine decrease when used in combination with abatacept?",
            "content": "",
            "analysis": "From the given context, it can be concluded that the therapeutic efficacy of Moderna COVID-19 Vaccine does decrease when used with abatacept",
            "final_answer": "yes"
        },
    ],

}

In [ ]:
# Define Few-shots prompt
fs_template = PromptTemplate(
    input_variables=["question", "content", "analysis", "final_answer"],
    template="Question: {question}\nContent: {content}\nAnalysis: {analysis}\nFinal Answer: {final_answer}"
)

In [ ]:
# Define few-shot prompt template
def fs_prompt_temp(knowledge_set):
  fs_p = FewShotPromptTemplate(
      examples=FS_EXAMPLES[knowledge_set],
      example_prompt=fs_template,
      suffix="Question: {input_question}\nContent: {input_content}",
      input_variables=["input_question", "input_content"]
  )
  return fs_p

#### Few shots generater for chat conversation

In [ ]:
def generate_fs_prompt_chat(knowledge_set):
  knowledge_set = FS_EXAMPLES[knowledge_set]
  fs_messages = []
  for example in knowledge_set:
    user_message = {'role': 'user', 'content': f"Question: {example['question']}\nContent: {example['content']}"}
    assistant_message = {'role': 'assistant', 'content': f"\nAnalysis: {example['analysis']}\nFinal Answer: {example['final_answer']}"}
    fs_messages.extend([user_message, assistant_message])
  return fs_messages

### Final Prompt

In [ ]:
def generate_final_prompt(knowledge_set, input_question, input_content):
  fs_prompt = fs_prompt_temp(knowledge_set)
  fs_prompt_input = fs_prompt.format(input_question=input_question, input_content=input_content)
  sys_prompt_input = generate_sys_prompt(knowledge_set)
  return [sys_prompt_input, fs_prompt_input]

In [ ]:
prompt_template = prompt_input = PromptTemplate(
    input_variables=["sys_prompt", "fs_prompt"],
    template="{sys_prompt}\n{fs_prompt}"
)
sys_prompt, fs_prompt = generate_final_prompt(knowledge_set='moderna_interactions', input_question='hi', input_content='yo')
print(prompt_template.format(sys_prompt=sys_prompt, fs_prompt=fs_prompt))

Task: You are tasked to answer yes/no questions based on the provided content about whether the therapeutic efficacy of Moderna COVID-19 Vaccine decrease when used with another drug. Understand the question, extract relevant information, analyze it, and provide a concise and accurate answer.
Answer Format: 
Analysis: Provide an analysis that logically leads to the answer based on the relevant information.
Final Answer: Provide the final answer. The answer should be 'yes' or 'no'.
        
Not to dos: Do not make assumptions not supported by the content. Avoid providing personal opinions or interpretations. Summarize and interpret the information as objectively and accurately as possible. You are providing an analysis, not diagnosing or treating medical conditions.
----Below are some examples-----

Question: Does the therapeutic efficacy of Moderna COVID-19 Vaccine decrease when used in combination with abatacept?
Content: The therapeutic efficacy of Moderna COVID-19 Vaccine decrease wh

## Parsers


#### binary_parser

In [ ]:
def binary_parser(output):
    sections = output.split('\n')
    analysis = ""
    final_answer = ""
    for section in sections:
        section_lower = section.lower()
        if section_lower.startswith("analysis: "):
            analysis = section_lower.replace("analysis: ", "")
        elif section_lower.startswith("final answer: "):
            final_answer = section_lower.replace("final answer: ", "").strip()
    # Ensuring that we only get 'yes' or 'no'
    if 'yes' in final_answer:
        final_answer = 'yes'
    elif 'no' in final_answer:
        final_answer = 'no'
    else:
        final_answer = ""
    return analysis, final_answer


#### mc_parser

In [ ]:
def mc_parser(output):
    sections = output.split('\n')
    analysis = ""
    final_answer = ""
    for section in sections:
        section_lower = section.lower()
        if section_lower.startswith("analysis: "):
            analysis = section_lower.replace("analysis: ", "")
        elif section_lower.startswith("final answer: "):
            final_answer = section_lower.replace("final answer: ", "").strip()
    # We only keep the first character of the final answer which should be the letter representing the chosen option
    final_answer = final_answer[0] if final_answer and final_answer[0].isalpha() else ""
    return analysis, final_answer


#### text_parser

In [ ]:
def text_parser(output):
    sections = output.split('\n')
    analysis = ""
    final_answer = ""
    for section in sections:
        section_lower = section.lower()
        if section_lower.startswith("analysis: "):
            analysis = section_lower.replace("analysis: ", "")
        elif section_lower.startswith("final answer: "):
            final_answer = section_lower.replace("final answer: ", "").strip()
    return analysis, final_answer


#### parser_dict

In [ ]:
# knowledge_paths
parser_dict = {
    'pubmedqa': binary_parser,
    'ade': text_parser,
    'chatDoctor': text_parser,
    'DDI_binary': binary_parser,
    'drug_usage': text_parser,
    'medmcqa': mc_parser,
    'mmlu_mc': mc_parser,
    'usmle_mc': mc_parser,
    'moderna_interactions': binary_parser,
}

## LLM Chain

In [ ]:
llm_chain = LLMChain(prompt=prompt_template, llm=llm)

NameError: ignored

#### Sanity Check

In [ ]:
test_question = 'Is methotrexate an alternative to azathioprine in neuromyelitis optica spectrum disorders with aquaporin-4 antibodies?'
test_content = ''
test_sys_prompt, test_fs_prompt = generate_final_prompt(knowledge_set='pubmedqa', input_question=test_question, input_content=test_content)
result = llm_chain.run({"sys_prompt": test_sys_prompt, "fs_prompt": test_fs_prompt})
analysis, final_answer = parser(result)
print("Analysis:\n", analysis)
print("Final Answer:\n", final_answer)

## Data Pre-processing

### Load Data

In [ ]:
def load_data(key, data_paths_dict, threshold):
    # Get the path of the data based on the key
    data_path = data_paths_dict[key]['data']

    # Load the data into a DataFrame
    df = pd.read_csv(data_path)
    if len(df) > threshold:
      df = df[:threshold]

    # Convert DataFrame to dict
    data_dict = {
        'id': df['id'].tolist(),
        'question': df['question'].tolist(),
        'context': df['context'].tolist(),
        'answer': df['answer'].tolist(),
    }

    n_samples = len(data_dict['question'])
    print(f"We have {n_samples} data samples in {key} dataset")

    return data_dict


In [ ]:
data_df_dict = {}
knowledge_df_dict = {
    'text': {},
    'embedding': {},
}
for key in data_paths_dict.keys():
    if key in ['medmcqa', 'mmlu_mc', 'usmle_mc', 'DDI_binary', 'drug_usage', 'chatDoctor', 'moderna_interactions']:
      knowledge_df_dict['text'][key] = pd.read_csv(knowledge_blocks_dict[key])
    if not key in ['DDI_binary', 'drug_usage', 'moderna_interactions']:
      data_df_dict[key] = load_data(key, data_paths_dict, SLICE_THRESHOLD)
    else:
      data_path = data_paths_dict[key]['data']
      # Load the data into a DataFrame
      df = pd.read_csv(data_path)
      # Convert DataFrame to dict
      data_dict = {
          'question': df['question'].tolist(),
          'answer': df['answer'].tolist(),
      }
      n_samples = len(data_dict['question'])
      data_df_dict[key] = data_dict
      print(f"We have {n_samples} data samples in {key} dataset")

We have 5000 data samples in pubmedqa dataset
We have 5000 data samples in ade dataset
We have 796 data samples in chatDoctor dataset
We have 600 data samples in DDI_binary dataset
We have 2888 data samples in drug_usage dataset
We have 2000 data samples in medmcqa dataset
We have 956 data samples in mmlu_mc dataset
We have 1499 data samples in usmle_mc dataset
We have 200 data samples in moderna_interactions dataset


### Dataloader

In [ ]:
def dataloader(key, data_df_dict):
    # Get the dictionary corresponding to the key
    data = data_df_dict[key]

    # Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

    # Shuffle the DataFrame
    df = df.sample(frac=1).reset_index(drop=True)

    # Decide the sample size based on the size of the dataset
    sample_size = EVALUATION_SIZE if len(df) > EVALUATION_SIZE else len(df)

    if data_paths_dict[key]['type'] == 'binary':
        min_samples = min(sample_size//2, min(df['answer'].value_counts().values))
        # For binary, ensure equal number of 'yes' and 'no' samples
        yes_df = df[df['answer'] == 'yes'].sample(n=min_samples, replace=False)
        no_df = df[df['answer'] == 'no'].sample(n=min_samples, replace=False)
        sample_df = pd.concat([yes_df, no_df])
        print(f"we have {len(yes_df)} of yes and {len(no_df)} of no in the loaded {key} evaluation set")
    elif data_paths_dict[key]['type'] == 'mc':
        min_samples = min(sample_size//4, min(df['answer'].value_counts().values))
        print(min_samples)
        # For multiple choice, ensure equal number of each option
        option_a_df = df[df['answer'] == 'A'].sample(n=min_samples, replace=False)
        option_b_df = df[df['answer'] == 'B'].sample(n=min_samples, replace=False)
        option_c_df = df[df['answer'] == 'C'].sample(n=min_samples, replace=False)
        option_d_df = df[df['answer'] == 'D'].sample(n=min_samples, replace=False)
        print(f"we have {len(option_a_df)} of A and {len(option_b_df)} of B and {len(option_c_df)} of C and {len(option_d_df)} of D in the loaded {key} evaluation set")
        sample_df = pd.concat([option_a_df, option_b_df, option_c_df, option_d_df])
    else:
        # Select a random sample of rows from the DataFrame
        sample_df = df.sample(n=sample_size)
        print(f"we have {len(sample_df)} of samples")
    # Convert the selected sample to a dict
    sample_data = {
        'sample': sample_df['question'].tolist(),
        'label': sample_df['answer'].tolist(),
    }

    return sample_data


In [ ]:
pubmed_eval = dataloader('pubmedqa', data_df_dict)
medmcqa_eval = dataloader('medmcqa', data_df_dict)
moderna_eval = dataloader('moderna_interactions', data_df_dict)

we have 500 of yes and 500 of no in the loaded pubmedqa evaluation set
250
we have 250 of A and 250 of B and 250 of C and 250 of D in the loaded medmcqa evaluation set
we have 100 of yes and 100 of no in the loaded moderna_interactions evaluation set


In [ ]:
print(pubmed_eval['sample'][0])
print(pubmed_eval['label'][0])

Does vasopressin improve survival compared with epinephrine in a neonatal piglet model of asphyxial cardiac arrest?
yes


## OPENAI API Handlers

In [ ]:
# Function to create a chat message and get a response
def chat_handler(query, prompt, few_shots, model):
    if USEOPENAI:
      while True:
        try:
            response = openai.ChatCompletion.create(
                messages = [
                    {'role': 'system', 'content': prompt},
                    *few_shots,
                    {'role': 'user', 'content': query},
                ],
                model=model,
                temperature=0.1,
            )
            return response['choices'][0]['message']['content']
        except Exception as e:
            print(f"Chat API error: {e}")
            continue


## Chat Implementation (set local to true to evaluate other models, current evaluating openAI models)

In [ ]:
def ask(query, model, knowledge_key, embedding_process, useknowledge, local):
    CoT = ''
    chat_response = ''

    # we can switch from using local open-source model or openAI api
    if local:

      # generate system prompt, few-shot prompt with user query and knowledge
      sys_prompt, fs_prompt = generate_final_prompt(input_question=query, input_content='')
      response = model.run({"sys_prompt": sys_prompt, "fs_prompt": fs_prompt})

      # parse the final result
      final_answer = response
      CoT = analysis
      chat_response = final_answer

    else:
      sys_prompt = generate_sys_prompt(knowledge_key)
      fs_prompt = generate_fs_prompt_chat(knowledge_key)
      # construct user query with domain-specific knowledge
      query = f"\nQuestion: {query} \nContent: {''}"
      # Initiate conversation
      response = chat_handler(query, sys_prompt, fs_prompt, model)

      final_answer = response
      CoT = analysis
      chat_response = final_answer

    return [chat_response, CoT, response]

## Evaluation

In [ ]:
dataset_name = 'mmlu_mc'
evaluation_set = dataloader(dataset_name, data_df_dict)

190
we have 190 of A and 190 of B and 190 of C and 190 of D in the loaded mmlu_mc evaluation set


In [ ]:
print(evaluation_set['sample'][0])
print(evaluation_set['label'][0])

A 35-year-old man comes to the office because of 1-week history of mid low back pain that radiates down his right leg. The pain began after the patient lifted a heavy box onto his truck. He rates his current pain as an 8 on a 10-point scale. He has been unable to find a comfortable position and has been sleeping in a recliner. Medical history is unremarkable and he takes no medications. He has smoked one pack of cigarettes daily for the past 25 years, and he drinks a six-pack of beer on Friday and Saturday nights. BMI is 27 kg/m2 . He appears uncomfortable and stands during the physical examination. Vital signs are normal. Straight-leg raise test is positive on the right, with loss of right ankle reflex. The remainder of the physical examination discloses no abnormalities. Which of the following is the most likely explanation for this patient’s symptoms? A:Displacement of the nucleus pulposus, B:Hypertrophy of the facet joints, C:Osteophyte formation, D:Spondylolisthesis
A


### metrics

In [ ]:
def check_text_accuracy(prediction, actual):
    # Remove any trailing periods from the prediction
    prediction = prediction.rstrip('.')

    # Generate the initials from the prediction
    prediction_initials = ''.join(word[0] for word in prediction.split())

    actual_words = actual.split()
    prediction_contains_all_words = all(word in prediction for word in actual_words)

    if prediction_contains_all_words or prediction_initials == actual:
        # Prediction contains all words in actual or initials match
        return True

    return False

In [ ]:
def calculate_f1_metrics(prediction, label):
    # Convert predictions and labels to sets for easy comparison
    prediction_set = set(prediction.split(', '))
    label_set = set(label.split(', '))
    print(prediction_set)
    print(label_set)
    # Compute true positives, false positives, and false negatives
    true_positives = len(prediction_set & label_set)
    false_positives = len(prediction_set - label_set)
    false_negatives = len(label_set - prediction_set)

    # Compute precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1_score


### GPT-3.5 Evaluation

#### Without CoT

In [ ]:
accurate_predictions = 0;
print_process = True;
basic_mc_prompt = 'Provide the final answer, which should be a single letter in the alphabet representing the best option among the multiple choices provided in the question. Follow this format:\nFinal Answer: A'
basic_binary_prompt = 'Provide the final answer, which should be a yes or no. Never say you dont know, always provide a yes or no answer regardless. If you dont have enough information, just guess. Follow this format:\nFinal Answer: yes'
for i in tqdm(range(len(evaluation_set['sample']) ), desc="Processing"):
  chat_response = chat_handler(evaluation_set['sample'][i], basic_binary_prompt, [], GPT_3)
  analysis, prediction = binary_parser(chat_response)
  if dataset_name == 'ade':
    if check_text_accuracy(prediction, evaluation_set['label'][i].lower()):
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} Question: {evaluation_set['sample'][i]} Chat Response: {prediction}, Answer: {evaluation_set['label'][i]}")
  else:
    prediction = chat_response.split(".")[0].lower()
    if prediction == evaluation_set['label'][i].lower():
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} Chat Response: {chat_response}, Answer: {evaluation_set['label'][i]}")
print(f"\nAccuracy: {accurate_predictions}/{len(evaluation_set['sample']) }")

#### With CoT

In [ ]:
accurate_predictions = 0;
print_process = True;
basic_binary_prompt = 'Analyze the question first, Provide the final answer, which should be a yes or no. Never say you dont know, always provide a yes or no answer regardless. If you dont have enough information, just guess. You must Follow this format:\nAnalysis: your analysis\nFinal Answer: no'
basic_mc_prompt = 'Analyze the question first, then provide the final answer, which should be a single letter in the alphabet representing the best option among the multiple choices provided in the question. Follow this format:\nAnalysis: your analysis\nFinal Answer: A'
basic_text_prompt = 'Analyze the question first, then provide the final answer, which should be a single term consisting of english words. Follow this format:\nAnalysis: your analysis\nFinal Answer: the adverse drug reaction you identified'
basic_double_binary_prompt = 'Your task is to answer to questions. Analyze the questions first, then provide the final answer, which should consist of two terms, each being either "yes" or "no" corresponding to the questions. Never say you dont know, always provide a yes or no answer regardless. If you dont have enough information, just guess. Follow this format:\n\nAnalysis: your analysis\nFinal Answer: no, no'
basic_text_list_prompt = '''Your task is to identify a list of medications. Analyze the question first, then provide the final answer, which should consist of drug names, each separated by a comma. Never say you don't know, always provide an answer regardless. If you don't have enough information, just guess. Follow this format:
Analysis: your analysis
Final Answer: DrugA, DrugB, DrugC, ...
'''
CoT_list = []
QA_list = []
precision_list = []
recall_list = []
f1_list = []
for i in tqdm(range(len(evaluation_set['sample'])), desc="Processing"):
  chat_response = chat_handler(evaluation_set['sample'][i], basic_binary_prompt, [], GPT_3)
  analysis, prediction = binary_parser(chat_response)
  CoT_list.append(chat_response)
  QA_list.append(prediction)
  if dataset_name == 'ade':
    if check_text_accuracy(prediction, evaluation_set['label'][i].lower()):
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} Question: {evaluation_set['sample'][i]} Chat Response: {prediction}, Answer: {evaluation_set['label'][i]}")
  elif dataset_name == 'drug_usage':
      # Extract the binary yes/no answers from the chat response
      answer1, answer2 = prediction.split(", ")
      label1, label2 = evaluation_set['label'][i].lower().split(", ")
      # Check the accuracy of each part of the answer
      if answer1.lower() == label1 and answer2.lower() == label2:
        accurate_predictions += 1
      else:
        print(f"\nWrong Answer Index: {i} Question: {evaluation_set['sample'][i]} Chat Response: {chat_response}, Answer: {evaluation_set['label'][i]}")
  elif dataset_name == 'chatDoctor':
      # Extract the binary yes/no answers from the chat response
        precision, recall, f1 = calculate_f1_metrics(prediction, evaluation_set['label'][i].lower())
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
  else:
    if prediction == evaluation_set['label'][i].lower():
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} YO Chat Response: {prediction}, Answer: {evaluation_set['label'][i]}")

data = {
      'Questions': evaluation_set['sample'],
      'Long Answer': CoT_list,
      'Short Answer': QA_list,
      'Official Answer': evaluation_set['label']
    }
# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a csv file
df.to_csv(data_paths_dict[dataset_name]['GPT_3_answer'], index=False)
print(f"\nAccuracy: {accurate_predictions}/{len(evaluation_set['sample']) }")

Processing:   1%|          | 2/200 [00:05<09:05,  2.75s/it]


Wrong Answer Index: 1 YO Chat Response: no, Answer: yes


Processing:   2%|▎         | 5/200 [00:15<09:41,  2.98s/it]


Wrong Answer Index: 4 YO Chat Response: no, Answer: yes


Processing:   3%|▎         | 6/200 [00:18<09:42,  3.00s/it]

...


Wrong Answer Index: 196 YO Chat Response: yes, Answer: no


Processing: 100%|█████████▉| 199/200 [25:28<00:02,  2.95s/it]


Wrong Answer Index: 198 YO Chat Response: yes, Answer: no


Processing: 100%|██████████| 200/200 [25:31<00:00,  7.66s/it]


Accuracy: 132/200


#### Without CoT

In [ ]:
accurate_predictions = 0;
print_process = True;
basic_mc_prompt = 'Provide the final answer, which should be a single letter in the alphabet representing the best option among the multiple choices provided in the question. Follow this format:\nFinal Answer: A'
basic_binary_prompt = 'Provide the final answer, which should be a yes or no. Never say you dont know, always provide a yes or no answer regardless. If you dont have enough information, just guess. Follow this format:\nFinal Answer: yes'
for i in tqdm(range(len(evaluation_set['sample']) ), desc="Processing"):
  chat_response = chat_handler(evaluation_set['sample'][i], basic_binary_prompt, [], GPT_4)
  analysis, prediction = binary_parser(chat_response)
  if dataset_name == 'ade':
    if check_text_accuracy(prediction, evaluation_set['label'][i].lower()):
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} Question: {evaluation_set['sample'][i]} Chat Response: {prediction}, Answer: {evaluation_set['label'][i]}")
  else:
    prediction = chat_response.split(".")[0].lower()
    if prediction == evaluation_set['label'][i].lower():
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} Chat Response: {chat_response}, Answer: {evaluation_set['label'][i]}")
print(f"\nAccuracy: {accurate_predictions}/{len(evaluation_set['sample']) }")

250
we have 250 of A and 250 of B and 250 of C and 250 of D in the loaded medmcqa evaluation set


Processing:   1%|          | 8/1000 [00:11<23:18,  1.41s/it]


Wrong Answer Index: 7 Chat Response: Final Answer: B, Answer: A


Processing:   3%|▎         | 27/1000 [00:37<20:06,  1.24s/it]


Wrong Answer Index: 26 Chat Response: Final Answer: C, Answer: A


...


Wrong Answer Index: 991 Chat Response: Final Answer: C, Answer: D


Processing:  99%|█████████▉| 993/1000 [35:10<00:07,  1.04s/it]


Wrong Answer Index: 992 Chat Response: Final Answer: B, Answer: D


Processing: 100%|██████████| 1000/1000 [35:18<00:00,  2.12s/it]


Accuracy: 802/1000


#### With CoT

In [ ]:
accurate_predictions = 0;
print_process = True;
basic_binary_prompt = 'Analyze the question first, Provide the final answer, which should be a yes or no. Never say you dont know, always provide a yes or no answer regardless. If you dont have enough information, just guess. Follow this format:\n Analysis: your analysis\nFinal Answer: no'
basic_mc_prompt = 'Analyze the question first, then provide the final answer, which should be a single letter in the alphabet representing the best option among the multiple choices provided in the question. Follow this format:\nAnalysis: your analysis\nFinal Answer: A'
basic_text_prompt = 'Analyze the question first, then provide the final answer, which should be a single term consisting of english words. Follow this format:\nAnalysis: your analysis\nFinal Answer: the adverse drug reaction you identified'
basic_double_binary_prompt = 'Your task is to answer to questions. Analyze the questions first, then provide the final answer, which should consist of two terms, each being either "yes" or "no" corresponding to the questions. Never say you dont know, always provide a yes or no answer regardless. If you dont have enough information, just guess. Follow this format:\n\nAnalysis: your analysis\nFinal Answer: no, no'
basic_text_list_prompt = '''Your task is to identify a list of medications. Analyze the question first, then provide the final answer, which should consist of drug names, each separated by a comma. Never say you don't know, always provide an answer regardless. If you don't have enough information, just guess. Follow this format:
Analysis: your analysis
Final Answer: DrugA, DrugB, DrugC, ...
'''
CoT_list = []
QA_list = []
precision_list = []
recall_list = []
f1_list = []
for i in tqdm(range(len(evaluation_set['sample'])), desc="Processing"):
  chat_response = chat_handler(evaluation_set['sample'][i], basic_binary_prompt, [], GPT_4)
  analysis, prediction = text_parser(chat_response)
  CoT_list.append(chat_response)
  QA_list.append(prediction)
  if dataset_name == 'ade':
    if check_text_accuracy(prediction, evaluation_set['label'][i].lower()):
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} Question: {evaluation_set['sample'][i]} Chat Response: {prediction}, Answer: {evaluation_set['label'][i]}")
  elif dataset_name == 'drug_usage':
      # Extract the binary yes/no answers from the chat response
      answer1, answer2 = prediction.split(", ")
      label1, label2 = evaluation_set['label'][i].lower().split(", ")
      # Check the accuracy of each part of the answer
      if answer1.lower() == label1 and answer2.lower() == label2:
        accurate_predictions += 1
      else:
        print(f"\nWrong Answer Index: {i} Question: {evaluation_set['sample'][i]} Chat Response: {chat_response}, Answer: {evaluation_set['label'][i]}")
  elif dataset_name == 'chatDoctor':
      # Extract the binary yes/no answers from the chat response
        precision, recall, f1 = calculate_f1_metrics(prediction, evaluation_set['label'][i].lower())
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
  else:
    if prediction == evaluation_set['label'][i].lower():
      accurate_predictions += 1
    else:
      print(f"\nWrong Answer Index: {i} Chat Response: {prediction}, Answer: {evaluation_set['label'][i]}")

data = {
      'Questions': evaluation_set['sample'],
      'Long Answer': CoT_list,
      'Short Answer': QA_list,
      'Official Answer': evaluation_set['label']
    }
# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a csv file
df.to_csv(data_paths_dict[dataset_name]['GPT_4_answer'], index=False)
print(f"\nAccuracy: {accurate_predictions}/{len(evaluation_set['sample']) }")

Processing:   1%|          | 2/200 [00:14<24:41,  7.48s/it]


Wrong Answer Index: 1 Chat Response: no, Answer: yes


Processing:   2%|▏         | 3/200 [00:20<21:13,  6.46s/it]


Wrong Answer Index: 2 Chat Response: no, Answer: yes


Processing:   2%|▏         | 4/200 [00:25<20:07,  6.16s/it]

...


Wrong Answer Index: 179 Chat Response: yes, Answer: no


Processing:  94%|█████████▍| 189/200 [25:48<01:13,  6.69s/it]


Wrong Answer Index: 188 Chat Response: yes, Answer: no


Processing: 100%|██████████| 200/200 [26:56<00:00,  8.08s/it]


Accuracy: 113/200
